# Install dependencies - Done

The `unicodecsv` library is needed for reading and manipulating CSV data. Since student enrollment info is in a CSV, we will need this library.

In [74]:
# Uncomment and run the following lines of code to install dependencies, if not already installed
#!pip install unicodecsv
#!pip install numpy

You should consider upgrading via the 'c:\users\alimd\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



## Load Data from CSVs - Done

We have the following csv files containing student data.

1. enrollments.csv
2. daily_engagement.csv
3. project_submissions.csv

In this section, each of these CSV files is loaded into the code, and stored in Python variables.

In [11]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()

with open('enrollments.csv', 'rb') as f:
    reader = unicodecsv.DictReader(f)    enroollments = list(reader)

In [12]:
#####################################
#                 1                 #
#####################################

## Read in the data from daily_engagement.csv and project_submissions.csv 
## and store the results in the below variables.
## Then look at the first row of each table.

def csv_as_list(filename):
    with open(filename, 'rb') as f:
        csvList = list(unicodecsv.DictReader(f))
    return csvList

daily_engagement = csv_as_list('daily_engagement.csv')
print(daily_engagement[0])
project_submissions = csv_as_list('project_submissions.csv')
print(project_submissions[0])

{'acct': '0', 'utc_date': '2015-01-09', 'num_courses_visited': '1.0', 'total_minutes_visited': '11.6793745', 'lessons_completed': '0.0', 'projects_completed': '0.0'}
{'creation_date': '2015-01-14', 'completion_date': '2015-01-16', 'assigned_rating': 'UNGRADED', 'account_key': '256', 'lesson_key': '3176718735', 'processing_state': 'EVALUATED'}


## Fixing Data Types - Done

The second part of Data Wrangling - that is, _Data Cleaning_ - phase begins here.

In [13]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{'account_key': '448',
 'status': 'canceled',
 'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 'days_to_cancel': 65,
 'is_udacity': True,
 'is_canceled': True}

In [108]:
# Commenting out the following block as it was purely for debugging
# debug_daily_engagement = csv_as_list('daily_engagement.csv')
# extra_minutes = []
# for record in debug_daily_engagement:
#     print(record['total_minutes_visited'], end=', ')
#     minutes = float(record['total_minutes_visited'])
#     if minutes >= 24 * 60 or str(minutes) != record['total_minutes_visited']:
#         extra_minutes.append(minutes)
#         print(minutes)
# print(extra_minutes)

In [14]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{'acct': '0',
 'utc_date': datetime.datetime(2015, 1, 9, 0, 0),
 'num_courses_visited': 1,
 'total_minutes_visited': 11.6793745,
 'lessons_completed': 0,
 'projects_completed': 0}

In [15]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 'assigned_rating': 'UNGRADED',
 'account_key': '256',
 'lesson_key': '3176718735',
 'processing_state': 'EVALUATED'}

## Problems in the Data - Done

**Note**: I've switched the order of blocks 2 and 3 because the changes in block 3 are needed in some wrangling of block 2.

In [16]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".
for engagement in daily_engagement:
    temp = engagement['acct']
    del engagement['acct']
    engagement['account_key'] = temp

## Investigating the Data - Done

In [17]:
#####################################
#                 2                 #
#####################################

## Find the total number of rows and the number of unique students (account keys)
## in each table.

def investigate(csvList):
    csvListLen = len(csvList)
    unique_elems = len(set([list_item['account_key'] for list_item in csvList]))
    return csvListLen, unique_elems

enrollment_num_rows, enrollment_num_unique_students = investigate(enrollments)

engagement_num_rows, engagement_num_unique_students = investigate(daily_engagement)

submission_num_rows, submission_num_unique_students = investigate(project_submissions)

print (enrollment_num_unique_students, engagement_num_unique_students, submission_num_unique_students)

1302 1237 743


In [19]:
print(daily_engagement[1000]['account_key'])

6


## Missing Engagement Records - Done

So in the above output, the enrollment data is 1302 but engagement data is 1237.
Meaning there are students enrolled not having any engagement. Find those students out!

In [20]:
#####################################
#                 4                 #
#####################################

## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.
engagement_keys = list(set([engagement['account_key'] for engagement in daily_engagement]))
enrolled_students_without_engagement = [student for student in enrollments if student['account_key'] not in engagement_keys]

In [21]:
print(enrolled_students_without_engagement[0])

{'account_key': '1219', 'status': 'canceled', 'join_date': datetime.datetime(2014, 11, 12, 0, 0), 'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), 'days_to_cancel': 0, 'is_udacity': False, 'is_canceled': True}


### Explanation
We see there are students who cancelled the same day they enrolled in, which is why they appear in the enrollments table, but not in the engagements table.

To see just how many of such students are there, run the following command.

In [22]:
print(len(enrolled_students_without_engagement))

71


## Checking for More Problem Records - Done

In [23]:
#####################################
#                 5                 #
#####################################

## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.
students_staying_one_day_min = []
for student in enrollments:
    if student['account_key'] not in engagement_keys and student['days_to_cancel'] is None:
        students_staying_one_day_min.append(student)
    elif student['account_key'] not in engagement_keys and student['days_to_cancel'] is not None and student['days_to_cancel'] > 0:
        students_staying_one_day_min.append(student)

In [24]:
print(len(students_staying_one_day_min))

3


In [25]:
print(students_staying_one_day_min)

[{'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 1, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), 'days_to_cancel': 59, 'is_udacity': True, 'is_canceled': True}, {'account_key': '1304', 'status': 'canceled', 'join_date': datetime.datetime(2015, 3, 10, 0, 0), 'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), 'days_to_cancel': 99, 'is_udacity': True, 'is_canceled': True}, {'account_key': '1101', 'status': 'current', 'join_date': datetime.datetime(2015, 2, 25, 0, 0), 'cancel_date': None, 'days_to_cancel': None, 'is_udacity': True, 'is_canceled': False}]


### Explanation

These students are all Udacity test accounts (we can tell since for each of them, `is_udacity` is `True`). There is no guarentee that Udacity test accounts will appear in enrollments.

## Tracking Down the Remaining Problems - Done

In [26]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [27]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [64]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print (len(non_udacity_enrollments))
print (len(non_udacity_engagement))
print (len(non_udacity_submissions))

1622
135656
3634


## Refining the Question - Done

Here, we ask the question this data analysis is suppose to answer. Additionally, we are now entering the Exploration Phase.

### Question
> How do the daily engagement attributes of students who pass their first project differ from those who don't?

### Problems
1. **`[High]`** The students are most likely to submit their projects during different intervals of time. A student might submit the first project by the end of the first week and another might submit it by the end of the first month, for example. These varying submission times will have varying engagement trends, and hence it is unfair to compare engagement across different ranges of time of students who submit.
2. **`[High]`** If you look at all submission info present in `daily_engagement` table, it contains engagement even after the submission of the first project. Those trends are of no use for the question posed, and would lead to incorrect analysis.
3. `[Low]` The total amount of info present in the `daily_engagement` table is for the entire course, and not just the project submissions. Hence you will be having a lot of data points that you won't be needing in answering the above question.

In order to tackle these problems, we start by eliminating the students who cancelled within the first week.

In [56]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = dict()
for student in non_udacity_enrollments:
    if student['cancel_date'] is None or student['days_to_cancel'] > 7:
        account_key = student['account_key']
        join_date = student['join_date']

        # add this entry to the dictionary only if it is already not there
        # and if it is the most recent enrollment
        if account_key not in paid_students or join_date > paid_students[account_key]:
            paid_students[account_key] = join_date

print(len(paid_students))

995


## Getting Data from First Week - Done

Next, to compare engagements across the same amount of time of students who have submitted, we extract engagement data in only the first week.

In [59]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

In [62]:
#####################################
#                 7                 #
#####################################

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = [
    engagement for engagement in non_udacity_engagement if engagement['account_key'] in paid_students and within_one_week(paid_students[engagement['account_key']], engagement['utc_date'])
]
# print(non_udacity_engagement[0])
print(paid_engagement_in_first_week[0])
print(len(paid_engagement_in_first_week))

{'utc_date': datetime.datetime(2015, 1, 9, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 11.6793745, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}
21508


In [102]:
for record in paid_engagement_in_first_week:
    minutes = 86400
    visited = float(record['total_minutes_visited'])
    visitNum = 0
    if visited >= minutes:
        visitNum += 1
        print(visited, end=", ")
print(f'visitnum: {visitNum}')
print(paid_engagement_in_first_week[0])

most = 0
for record in paid_engagement_in_first_week:
    if record['total_minutes_visited'] > most:
        most = record['total_minutes_visited']
print(f'most: {most}')


visitnum: 0
{'utc_date': datetime.datetime(2015, 1, 9, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 11.6793745, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}
most: 1030.88319667


In [66]:
# Now, to go along with the official solution, I would like to create the 3 variables I might need later, containing info of paid students.
def filter_out_unpaid_students(data):
    paid_data = [
        data_point for data_point in data if data_point['account_key'] in paid_students
    ]
    return paid_data

paid_enrollments = filter_out_unpaid_students(non_udacity_enrollments)
paid_engagement = filter_out_unpaid_students(non_udacity_engagement)
paid_submissions = filter_out_unpaid_students(non_udacity_submissions)

print(len(paid_enrollments))
print(len(paid_engagement))
print(len(paid_submissions))

1293
134549
3618


## Exploring Student Engagement - Done

In [68]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
engagement_by_account = defaultdict(list)
for engagement_record in paid_engagement_in_first_week:
    account_key = engagement_record['account_key']
    engagement_by_account[account_key].append(engagement_record)

In [71]:
print(engagement_by_account['0'])

[{'utc_date': datetime.datetime(2015, 1, 9, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 11.6793745, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 10, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 37.2848873333, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 11, 0, 0), 'num_courses_visited': 2, 'total_minutes_visited': 53.6337463333, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 12, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 33.4892696667, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 13, 0, 0), 'num_courses_visited': 1, 'total_minutes_visited': 64.7796776667, 'lessons_completed': 0, 'projects_completed': 0, 'account_key': '0'}, {'utc_date': datetime.datetime(2015, 1, 14, 0, 0), 'num_courses_visi

In [109]:
# Commenting out the following block as it was purely for debugging

# debug_engagement_by_account = defaultdict(list)
# for debug_engagement in paid_engagement_in_first_week:
#     account_key = engagement_record['account_key']
#     if debug_engagement not in debug_engagement_by_account['account_key']:
#         debug_engagement_by_account[account_key].append(debug_engagement)

In [110]:
# print(len(debug_engagement_by_account))
# print(len(engagement_by_account))

In [72]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)
total_minutes_by_account = {}
for account_key, engagement_for_student in engagement_by_account.items():
    total_minutes = 0
    for engagement_record in engagement_for_student:
        total_minutes += engagement_record['total_minutes_visited']
    total_minutes_by_account[account_key] = total_minutes

In [79]:
import numpy as np

# Summarize the data about minutes spent in the classroom
total_minutes = list(total_minutes_by_account.values())
print ('Mean:', np.mean(total_minutes))
print ('Standard deviation:', np.std(total_minutes))
print ('Minimum:', np.min(total_minutes))
print ('Maximum:', np.max(total_minutes))

Mean: 647.5901738262695
Standard deviation: 1129.2712104188108
Minimum: 0.0
Maximum: 10568.100867332541


In [107]:
# Commenting out the following block as it was purely for debugging

# most = 0
# for minutes in list(total_minutes_by_account.values()):
#     if minutes > most:
#         most = minutes
# print(most)

# most = 86400
# extra = []
# for minutes in list(total_minutes_by_account.values()):
#     if minutes > most:
#         extra.append(minutes)
# print(extra)
# print(len(extra))

10568.100867332541
[2530.6469816678004, 1591.3228143334, 2997.70812683204, 1579.12122666663, 1917.2210374995, 1898.9588304992003, 2783.6493991655298, 3933.373638167089, 2185.0847380002706, 4842.298387167651, 2098.8426373316, 3946.50200616513, 5879.230190166303, 7229.78709266389, 2917.4438696675406, 3656.38093583222, 3124.592313998191, 2293.9219658344996, 1867.74218416554, 2328.9144675007697, 10568.100867332541, 3991.9810373311407, 5807.212303334741, 1449.3736278338001, 7263.991842167299, 6709.42387016856, 2916.9502008342706, 5519.3081969980985, 3914.010639835101, 3564.7332644989997, 4962.5543161655005, 1887.391213833, 2375.7954986677, 2634.85551000113, 3862.929400999971, 4224.377395331539, 1810.5199383324698, 6027.846535335141, 3869.08683200167, 1452.709982333, 2111.75571399998, 2393.5421573319004, 2662.8567701651705, 3716.85055883333, 2884.7102329999, 5740.986651833372, 4752.494575498951, 4352.512844498472, 2040.99096683197, 2700.4878774989998, 1918.8834441663, 1735.943607166, 2778.31

## Debugging Data Analysis Code

In [ ]:
#####################################
#                 8                 #
#####################################

## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

## Lessons Completed in First Week

In [ ]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

## Number of Visits in First Week

In [ ]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.

## Splitting out Passing Students

In [ ]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.

subway_project_lesson_keys = ['746169184', '3176718735']

passing_engagement =
non_passing_engagement =

## Comparing the Two Student Groups

In [ ]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [ ]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [ ]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.